In [ ]:
# Copyright 2022 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import dataclasses
import itertools
import json
import os
import sys
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

In [ ]:
if 'gfile' not in sys.modules:
  gfile = tf.io.gfile

In [ ]:
PROJECT_ROOT = '/your/project/folder/here'
SUBDIR = 'your_subdir'

In [ ]:
MODELS = ['tgn', 'edgebank', 'tgn_structmap']
DATA = ['tgbl_flight;AS;AS;AF']
EXPERIMENTS = [
    'transductive',
    'transfer_no_warmstart',
    'transfer_warmstart',
]
MODELS_EXPERIMENTS = [
    'tgn-transfer_no_warmstart',
    'tgn-transfer_warmstart',
    'tgn_structmap-transfer_no_warmstart',
]

In [ ]:
# Transform inputs.
@dataclasses.dataclass(frozen=False)
class DatasetSpec:
  dataset: str = dataclasses.field(default_factory=str)
  train_split: str = dataclasses.field(default_factory=str)
  val_split: str = dataclasses.field(default_factory=str)
  test_split: str = dataclasses.field(default_factory=str)


@dataclasses.dataclass(frozen=False)
class ExperimentResults:
  experiment: str = dataclasses.field(default_factory=str)
  train_results: dict[str, float] = dataclasses.field(default_factory=dict)
  test_results: dict[str, float] = dataclasses.field(default_factory=dict)
  val_warmstart_loss_metrics: pd.DataFrame = dataclasses.field(
      default_factory=pd.DataFrame
  )
  val_loss_metrics: pd.DataFrame = dataclasses.field(
      default_factory=pd.DataFrame
  )
  test_warmstart_loss_metrics: pd.DataFrame = dataclasses.field(
      default_factory=pd.DataFrame
  )
  test_loss_metrics: pd.DataFrame = dataclasses.field(
      default_factory=pd.DataFrame
  )


@dataclasses.dataclass(frozen=False)
class ModelResults:
  model: str = dataclasses.field(default_factory=str)
  experiment_results: dict[str, ExperimentResults] = dataclasses.field(
      default_factory=dict
  )


@dataclasses.dataclass(frozen=False)
class DatasetResults:
  dataset: str = dataclasses.field(default_factory=str)
  model_results: dict[str, ModelResults] = dataclasses.field(
      default_factory=dict
  )


DATASET_SPECS = []
DATASETS = []
for dataset_string in DATA:
  dataset, train_split, val_split, test_split = dataset_string.split(';')
  DATASETS.append(dataset)
  DATASET_SPECS.append(DatasetSpec(dataset, train_split, val_split, test_split))

RESULTS_SUBDIR = os.path.join(PROJECT_ROOT, 'experiments', SUBDIR)

In [ ]:
ALL_RESULTS = {}
train_results_df = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [MODELS, EXPERIMENTS], names=['Model', 'Experiment']
    ),
    columns=pd.MultiIndex.from_product(
        [DATASETS, ['auc', 'mrr']], names=['Dataset', 'Metric']
    ),
)
test_results_df = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [MODELS, EXPERIMENTS], names=['Model', 'Experiment']
    ),
    columns=pd.MultiIndex.from_product(
        [DATASETS, ['auc', 'mrr']], names=['Dataset', 'Metric']
    ),
)
for dataset_spec in DATASET_SPECS:
  dataset_results = DatasetResults(dataset=dataset_spec.dataset)
  for model in MODELS:
    model_results = ModelResults(model=model)
    model_dataset_folder = os.path.join(
        RESULTS_SUBDIR,
        dataset_spec.dataset,
        'results',
        f'{model}_{dataset_spec.dataset}_{dataset_spec.train_split}_{dataset_spec.val_split}',
    )
    for experiment in EXPERIMENTS:
      if str(model) + '-' + str(experiment) in MODELS_EXPERIMENTS:
        # Extract results for train.
        experiment_results = ExperimentResults(experiment=experiment)
        with gfile.GFile(
            os.path.join(
                model_dataset_folder, f'{experiment}_results_train.json'
            ),
            'r',
        ) as f:
          experiment_results.train_results = json.load(f)
        with gfile.GFile(
            os.path.join(model_dataset_folder, f'{experiment}_val_loss.csv'),
            'r',
        ) as f:
          experiment_results.val_loss_metrics = pd.read_csv(f)
        if not 'no_warmstart' in experiment:
          with gfile.GFile(
              os.path.join(model_dataset_folder, f'{experiment}_val_loss.csv'),
              'r',
          ) as f:
            experiment_results.val_warmstart_loss_metrics = pd.read_csv(f)
        train_results_df.loc[model, experiment].at[
            dataset_spec.dataset, 'auc'
        ] = experiment_results.train_results['auc']
        train_results_df.loc[model, experiment].at[
            dataset_spec.dataset, 'mrr'
        ] = experiment_results.train_results['val mrr']

        # Extract results for test.
        with tf.io.gfile.GFile(
            os.path.join(
                model_dataset_folder,
                f'{experiment}_results_test_{dataset_spec.test_split}.json',
            ),
            'r',
        ) as f:
          experiment_results.test_results = json.load(f)
        with tf.io.gfile.GFile(
            os.path.join(
                model_dataset_folder,
                f'{experiment}_test_{dataset_spec.test_split}_loss.csv',
            ),
            'r',
        ) as f:
          experiment_results.test_loss_metrics = pd.read_csv(f)
        if not 'no_warmstart' in experiment:
          with tf.io.gfile.GFile(
              os.path.join(
                  model_dataset_folder,
                  f'{experiment}_test_{dataset_spec.test_split}_loss.csv',
              ),
              'r',
          ) as f:
            experiment_results.test_warmstart_loss_metrics = pd.read_csv(f)

        test_results_df.loc[model, experiment].at[
            dataset_spec.dataset, 'auc'
        ] = experiment_results.test_results['test auc']
        test_results_df.loc[model, experiment].at[
            dataset_spec.dataset, 'mrr'
        ] = experiment_results.test_results['test mrr']

        model_results.experiment_results[experiment] = experiment_results

    dataset_results.model_results[model] = model_results
  ALL_RESULTS[dataset_spec.dataset] = dataset_results

In [ ]:
import copy
from matplotlib import pyplot as plt
import seaborn as sns


def plot_eval_metric_curves(
    dataset='tgbl_wiki',
    experiments=EXPERIMENTS,
    models=['tgn', 'tgn_structmap'],
    model_experiemnts=MODELS_EXPERIMENTS,
    val=True,
    metric_name='perf',
    batches_plot=100,
    plot_warmstart_period=False,
    master_results_dict=ALL_RESULTS,
    dataset_spec=DATASET_SPECS,
):

  eval_df_string = 'val' if val else 'test'
  plot_dataframes = []

  # Make sure that if there is warmstart experiment that it comes first.
  experiment_list = copy.deepcopy(experiments)
  first_experiment = ''
  for experiment in experiment_list:
    if 'no_warmstart' not in experiment:
      first_experiment = copy.deepcopy(experiment)
  experiment_list.remove(first_experiment)
  experiment_list = [first_experiment] + experiment_list

  warmstart_end_index = 0
  for idx, experiment in enumerate(experiment_list):
    for model in models:
      if str(model) + '-' + str(experiment) in model_experiemnts:
        if ('no_warmstart' not in experiment) and plot_warmstart_period:
          warmstart_df = getattr(
              master_results_dict[dataset]
              .model_results[model]
              .experiment_results[experiment],
              f'{eval_df_string}_warmstart_loss_metrics',
          )
          if idx == 0:
            warmstart_end_index = len(warmstart_df)
          warmstart_df['batch_index'] = list(range(warmstart_end_index))
          warmstart_df = warmstart_df.melt(
              id_vars=['batch_index'],
              value_vars=['loss', 'perf', 'auc'],
              value_name='metric_value',
              var_name='metric_name',
          )
          warmstart_df['experiment'] = model + '-' + experiment
          warmstart_df = warmstart_df[warmstart_df.metric_name == metric_name]
          warmstart_df['period'] = 'warmstart'
          plot_dataframes.append(warmstart_df.head(batches_plot))

        eval_df = getattr(
            master_results_dict[dataset]
            .model_results[model]
            .experiment_results[experiment],
            f'{eval_df_string}_loss_metrics',
        )
        eval_df['batch_index'] = list(
            range(warmstart_end_index, len(eval_df) + warmstart_end_index)
        )
        eval_df = eval_df.melt(
            id_vars=['batch_index'],
            value_vars=['loss', 'perf', 'auc'],
            value_name='metric_value',
            var_name='metric_name',
        )
        eval_df['experiment'] = model + '-' + experiment
        eval_df = eval_df[eval_df.metric_name == metric_name]
        eval_df['period'] = 'eval'
        plot_dataframes.append(eval_df.head(batches_plot))

  # Make plot.
  master_plot_dataframe = pd.concat(plot_dataframes, axis=0)
  plt.figure(figsize=(15, 10))
  plt.title(f'{dataset} {eval_df_string} {metric_name}')
  sns.lineplot(
      data=master_plot_dataframe,
      x='batch_index',
      y='metric_value',
      hue='experiment',
      style='period',
  )
  plt.show()
  return master_plot_dataframe

In [ ]:
plot_df = plot_eval_metric_curves(
    dataset='tgbl_flight', metric_name='loss', batches_plot=60, val=False
)

In [ ]:
plot_df = plot_eval_metric_curves(
    dataset='tgbl_flight', metric_name='perf', batches_plot=100, val=False
)

In [ ]:
plot_df = plot_eval_metric_curves(
    dataset='tgbl_flight', metric_name='auc', batches_plot=100, val=False
)